# Import all dependencies

We import all the dependencies required to run the benchmark

In [ ]:
from lips.config.configmanager import ConfigManager
from lips.augmented_simulators.torch_models.u_net import TorchUnet
from lips.augmented_simulators.torch_models.fully_connected import TorchFullyConnected
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.sampler import LHSSampler
from lips.dataset.scaler.standard_scaler_per_channel import StandardScalerPerChannel
from lips.dataset.scaler.standard_scaler import StandardScaler

from lips.physical_simulator.getfemSimulator import GetfemSimulator
import lips.physical_simulator.GetfemSimulator.PhysicalFieldNames as PFN

from lips.benchmark.wheelBenchmark import WeightSustainingWheelBenchmark
from lips.dataset.pneumaticWheelDataSet import SamplerStaticWheelDataSet,DataSetInterpolatorOnGrid,DataSetInterpolatorOnMesh


# Constant definition

We defined the constants used all through the program

In [ ]:
CONFIG_PATH_BENCHMARK="/home/ddanan/HSAProject/LIPSPlatform/LIPS_Github/LIPS/configurations/pneumatic/benchmarks/confWheel.ini"

# Datasets generation

We generate the train/test and validation datasets

In [ ]:
wheel_config=ConfigManager(path=CONFIG_PATH_BENCHMARK,
                              section_name="WeightSustainingWheelBenchmarkRegular")

print(wheel_config)

In [ ]:
env_params=wheel_config.get_option("env_params")
physical_domain=env_params.get("physical_domain")
physical_properties=env_params.get("physical_properties")
simulator=GetfemSimulator(physical_domain=physical_domain,physical_properties=physical_properties)

# Sampler generation

In [ ]:
sampler=wheel_config.get_option("sampler")
sampler_input_params=sampler.get("sampler_input_params")
sampler_seeds=sampler.get("seeds")
sampler_nb_samples=sampler.get("nb_samples")

lhs_sampler=LHSSampler(space_params=sampler_input_params)
print(lhs_sampler)

# Training dataset generation

In [ ]:
attr_names=(PFN.displacement,)
DATA_PATH="WeightSustainingWheelBenchmarkRegular"

pneumatic_wheel_dataset_train=SamplerStaticWheelDataSet("train",attr_names=attr_names,config=wheel_config)
pneumatic_wheel_dataset_train.generate(simulator=simulator,
                                    actor=lhs_sampler,
                                    nb_samples=sampler_nb_samples.get("train"),
                                    actor_seed=sampler_seeds.get("train"),
                                    path_out=DATA_PATH
                                    )

In [ ]:
pneumatic_wheel_dataset_val=SamplerStaticWheelDataSet("val",attr_names=attr_names,config=wheel_config)
pneumatic_wheel_dataset_val.generate(simulator=simulator,
                                    actor=lhs_sampler,
                                    nb_samples=sampler_nb_samples.get("val"),
                                    actor_seed=sampler_seeds.get("val"),
                                    path_out=DATA_PATH
                                    )

In [ ]:
pneumatic_wheel_dataset_test=SamplerStaticWheelDataSet("test",attr_names=attr_names,config=wheel_config)
pneumatic_wheel_dataset_test.generate(simulator=simulator,
                                    actor=lhs_sampler,
                                    nb_samples=sampler_nb_samples.get("test"),
                                    actor_seed=sampler_seeds.get("test"),
                                    path_out=DATA_PATH
                                    )

# Start training with FC architecture

In [ ]:
LOG_PATH="WeightSustainingFFNN.log"
CONFIG_PATH_AUGMENTED_SIMULATOR_FC="/home/ddanan/HSAProject/LIPSPlatform/LIPS_Github/LIPS/configurations/pneumatic/simulators/torch_fc.ini"
DATA_PATH="."

benchmark_ffnn = WeightSustainingWheelBenchmark(benchmark_name="WeightSustainingWheelBenchmarkRegular",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=CONFIG_PATH_BENCHMARK
                               )

torch_sim = TorchSimulator(name="torch_ffnn",
                           model=TorchFullyConnected,
                           scaler=StandardScaler,
                           log_path=LOG_PATH,
                           seed=42,
                           architecture_type="Classical",
                           sim_config_path=CONFIG_PATH_AUGMENTED_SIMULATOR_FC
                          )

SAVE_PATH="/home/ddanan/HSAProject/LIPSPlatform/LIPS_Github/LIPS/getting_started/TestBenchmarkWheel/FFNNResults"
torch_sim_config=ConfigManager(path=CONFIG_PATH_AUGMENTED_SIMULATOR_FC,
                              section_name="CONFIGWHEELSUSTAIN")

In [ ]:
torch_sim.params

In [ ]:
torch_sim_params=torch_sim_config.get_options_dict()
torch_sim.train(train_dataset=benchmark_ffnn.train_dataset,
                    val_dataset=benchmark_ffnn.val_dataset,
                    save_path=SAVE_PATH,
                    **torch_sim_params)

In [ ]:
torch_sim_metrics_val = benchmark_ffnn.evaluate_simulator(augmented_simulator=torch_sim,
                                                  eval_batch_size=128,
                                                  dataset="val",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )

In [ ]:
torch_sim_metrics_test = benchmark_ffnn.evaluate_simulator(augmented_simulator=torch_sim,
                                                  eval_batch_size=128,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )

In [ ]:
torch_sim_metrics_val

# Start training with CNN

In [ ]:
wheel_config=ConfigManager(path=CONFIG_PATH_BENCHMARK,
                              section_name="WeightSustainingWheelBenchmarkInterpolated")
env_params=wheel_config.get_option("env_params")
physical_domain=env_params.get("physical_domain")
physical_properties=env_params.get("physical_properties")
simulator=GetfemSimulator(physical_domain=physical_domain,physical_properties=physical_properties)

attr_x= wheel_config.get_option("attr_x")
attr_y= ("disp",)
attr_names=attr_x+attr_y

In [ ]:
pneumatic_wheel_datasets={
    "train":pneumatic_wheel_dataset_train,
    "val":pneumatic_wheel_dataset_val,
    "test":pneumatic_wheel_dataset_test
    
}

# Define interpolation

In [ ]:
interpolation_info=wheel_config.get_option("interpolation_info")
grid_support=interpolation_info.get("grid_support")
dofnum_by_field=interpolation_info.get("dofnum_by_field")

In [ ]:
dataset_by_type=dict()
for name,dataset in pneumatic_wheel_datasets.items():
    myTransformer=DataSetInterpolatorOnGrid(name=name,
                                                simulator=simulator,
                                                dataset=dataset,
                                                grid_support=grid_support)
    myTransformer.generate(dofnum_by_field=dofnum_by_field,path_out="WeightSustainingWheelBenchmarkInterpolated")
    dataset_by_type[dataset.name]=myTransformer

In [ ]:
LOG_PATH="WeightSustainingCNN.log"
benchmark_cnn = WeightSustainingWheelBenchmark(benchmark_name="WeightSustainingWheelBenchmarkInterpolated",
                                benchmark_path=".",
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=CONFIG_PATH_BENCHMARK
                               )

In [ ]:
torch_sim = TorchSimulator(name="torch_unet",
                           model=TorchUnet,
                           scaler=StandardScalerPerChannel,
                           log_path=LOG_PATH,
                           seed=42,
                           architecture_type="Classical",
                          )

In [ ]:
SAVE_PATH="/home/ddanan/HSAProject/LIPSPlatform/LIPS_Github/LIPS/getting_started/TestBenchmarkWheel/CNNResults"
CONFIG_PATH_AUGMENTED_SIMULATOR_UNET="/home/ddanan/HSAProject/LIPSPlatform/LIPS_Github/LIPS/configurations/pneumatic/simulators/torch_unet.ini"
torch_sim_config=ConfigManager(path=CONFIG_PATH_AUGMENTED_SIMULATOR_UNET,
                              section_name="DEFAULT")
torch_sim_params=torch_sim_config.get_options_dict()

In [ ]:
torch_sim_params["epochs"]=2
torch_sim.train(train_dataset=benchmark_cnn.train_dataset,
                    val_dataset=benchmark_cnn.val_dataset,
                    save_path=SAVE_PATH,
                    **torch_sim_params)

In [ ]:
predictor_val = benchmark_cnn.evaluate_predictor(augmented_simulator=torch_sim,
                                                  eval_batch_size=128,
                                                  dataset="val",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )

In [ ]:
name="val"
field_name=PFN.displacement
prediction=predictor_val[name]
interpolated_field_name=field_name+"Interpolated"
prediction[field_name] = prediction.pop(interpolated_field_name)
simulator=dataset_by_type[name].simulator

# Interpolate back to mesh

In [ ]:
interpolated_dataset_grid=DataSetInterpolatorOnGrid(name=name,
                                                      simulator=simulator,
                                                      dataset=dataset_by_type[name],
                                                      grid_support=dataset_by_type[name].grid_support)

interpolated_dataset_grid.load_from_data(grid_support_points=dataset_by_type[name].grid_support_points,
                                           interpolated_dataset=prediction,
                                           distributed_inputs_on_grid=dataset_by_type[name].distributed_inputs_on_grid)

interpolated_datas_on_mesh=DataSetInterpolatorOnMesh(name=name,
                                                      simulator=simulator,
                                                      dataset=interpolated_dataset_grid)
interpolated_datas_on_mesh.generate(field_names=[field_name])
prediction_on_mesh={name: interpolated_datas_on_mesh.interpolated_dataset}
observation_val_on_mesh={"val":pneumatic_wheel_datasets["val"].data}

In [ ]:
torch_sim_metrics_val = benchmark_cnn.evaluate_simulator_from_predictions(predictions=prediction_on_mesh,
                                                                             observations=observation_val_on_mesh,
                                                                             eval_batch_size=128,
                                                                             dataset="val",
                                                                             shuffle=False,
                                                                             save_path=None,
                                                                             save_predictions=False
                                                                            )
print(torch_sim_metrics_val)